In [111]:
### to-do ###
# compare rates with electronic police report data where rtcc footage is not requested 

In [112]:
import pandas as pd

def clean():
    df1 = pd.read_csv("electronic_police_report_2018.csv")
    df2 = pd.read_csv("electronic_police_report_2019.csv")
    df3 = pd.read_csv("electronic_police_report_2020.csv")
    df4 = pd.read_csv("electronic_police_report_2021.csv")
    df5 = pd.read_csv("electronic_police_report_2022.csv")

    dfs = [df1, df2, df3, df4, df5]
    dfa = pd.concat(dfs, join="outer")
    
    dfb = pd.read_csv("rtcc.csv", encoding="cp1252").rename(columns={"Item_number": "item_number"})

    df = pd.merge(dfa, dfb, on="item_number")
    return df

In [113]:
df = clean

In [114]:
def drop_rows_missing_race(df):
    df.loc[:, "offender_race"] = df.offender_race.fillna("").str.lower().str.strip()\
    .str.replace("unknown", "", regex=False)
    # .str.replace("hispanic", "non-black", regex=False)\
    # .str.replace("asian", "non-black", regex=False)\
    # .str.replace("white", "non-black", regex=False)\
    # .str.replace(r"amer\. ind\.", "non-black", regex=True)
    return df[~((df.offender_race == ""))]


def filter_gender(df):
    df.loc[:, "offender_gender"] = (
        df.offender_gender.str.lower().str.strip().fillna("").str.replace("unknown", "", regex=False)
    )
    df = df[(df.offender_gender.isin(["male"]))]
    return df[~((df.offender_gender == ""))]


def filter_arrested(df):
    df.loc[:, "offenderstatus"] = df.offenderstatus.str.lower().str.strip().fillna("")
    df = df[df.offenderstatus.isin(["arrested"])]
    return df[~((df.offenderstatus == ""))]


def filter_year(df):
    df = df[df.year.astype(str).isin(["2018", "2019", "2020", "2021", "2022"])]
    # the following figures are percentages of the "offender_race" value
    ### 2018 - 2022
    """
      black         0.768292
      unknown       0.152651
      white         0.068554
      hispanic      0.009226
      asian         0.001065
      -amer. ind.    0.000213
    """
    ### 2022
    """
      black      0.660252
      unknown    0.297557
      white      0.039230
      asian      0.002961
    """
    ### 2021
    """
      black       0.663802
      unknown     0.279270
      white       0.055209
      asian       0.001074
      hispanic    0.000644
    """
    ### 2020
    """
      black         0.867308
      unknown       0.069408
      white         0.046369
      hispanic      0.016040
      amer. ind.    0.000875
    """
    ### 2019
    """
      black       0.849009
      white       0.091532
      unknown     0.044685
      hispanic    0.013694
      asian       0.001081
    """
    ### 2018
    """
      black       0.804891
      white       0.129187
      unknown     0.046252
      hispanic    0.018075
      asian       0.001595
    """
    return df


def extract_years(df):
    years = df.occurred_date_time.astype(str).str.extract(r"(\w{4})")

    df.loc[:, "year"] = years[0]
    return df

In [115]:
def filter_data():
  df = clean()
  df = df.pipe(filter_year).pipe(drop_rows_missing_race)
  return df

In [116]:
df = filter_data()

c:\Users\PC\AppData\Local\Programs\Python\Python39\lib\site-packages\IPython\core\interactiveshell.py:3398: DtypeWarning: Columns (21) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [117]:
df["rtcc_footage_request_tally"] = 1

In [118]:
df = df[["offender_race", "rtcc_footage_request_tally"]]
df = df.groupby(["offender_race"]).rtcc_footage_request_tally.count()
df = pd.DataFrame(df).reset_index()

In [119]:
df.loc[(df.offender_race == "black"), "population"] = "229000"
df.loc[(df.offender_race == "white"), "population"] = "120000"
df.loc[(df.offender_race == "hispanic"), "population"] = "21500"
df.loc[(df.offender_race == "amer. ind."), "population"] = "723"
df.loc[(df.offender_race == "asian"), "population"] = "11305"

In [120]:
def calc_rate(dfb_total, population=""):
    population = int(population)
    rate = (( dfb_total / population) * 1000)
    return rate

In [121]:
df["rate_per_1000_persons"] = df.apply(lambda x: calc_rate(x.rtcc_footage_request_tally, x.population), axis=1)

In [122]:
df

,offender_race,rtcc_footage_request_tally,population,rate_per_1000_persons
0,amer. ind.,3,723,4.149378
1,asian,15,11305,1.326847
2,black,10826,229000,47.275109
3,hispanic,130,21500,6.046512
4,white,966,120000,8.050000
